# RageSimulator
This simulator is written in rage against how awful the original Piccolissimo simulator is, and rewriting it. 

## Relevant Equations 

### Newton-Euler (equations 2.2 and 2.3)
$$\mathbf{\dot{v}^I_F} = (\mathbf{f_{aero}} + m \mathbf{g} - m \mathbf{\omega^{FI} }\times \mathbf{v^I_F})$$

$$\mathbf{\dot{\omega}^{FI}} = \mathbf{I}^{-1}(\mathbf{\tau_{aero}} - \mathbf{\omega^{FI} }\times \mathbf{I\omega^{BI}})$$

* $v_F^I$ is velocity of the center of mass of flyer frame in inertial/world frame
* $\mathbf{\omega^{FI} } = [p\ q\ r]^T$ is the angular velocity of flyer frame w.r.t. inertial
* $\mathbf{\omega^{BI} }$ is the angular velocity of body frame wrt inertial (same as flyer but with rotation about z axis).




### Blade Equations (equations 3.8)
$$[v_E^I]^E = \mathbf{R^E_F}[v^I_F]^F + \mathbf{R^E_F}[\omega^{SF}]^F \times \mathbf{R^E_F} [S_{EF}]^F + \mathbf{R^E_F}[\nu]^F$$
$$\gamma = arctan(\frac{[v_E^I]^E [\hat{z}]^E}{[v_E^I]^P [\hat{x}]^E})$$


* $[\nu]^F = [0\ 0\ \nu]^T$ is induced inflow velocity
* $\nu$ is the induced inflow velocity (Equation 3.1, thesis p.52)
* $[v_E^I]^E$ is the relative wing in the propeller frame.


### Euler Angles Convention
For euler angles, we are using the `scipy` library of `spatial.transform` with `Rotation`. https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html#scipy.spatial.transform.Rotation

This uses the capital notation `XYZ` to denote intrinsic rotations, which we will use such that the final rotation is about the rotational axis of the vehicle for readibility. This is such that the final rotation is about the Z axis, or corresponds to the in-body yaw. 

Please use the `euler2Rotm` and `rotm2Euler` functions for converting between rotation matricies and euler angle vectors 

## Code parameters

* camelCase unless stated otherwise
* Underscores for frames or for subscripts


## Notes
Flyer frame means that it is *not* in the frame of the propellor with the extra spin on the propellor. This is 
only relevant for the shaft propellor.

For the body drag plates, this is all in the body frame (no extra spin) so the euler angular rates are still correct, as well as the euler angles. 

TODO: read through the math extra carefully again. 
Be explicit with the notation and work through each sub example - write an example for
that one and the next one.

In [18]:
%matplotlib inline
import numpy as np 
from scipy.spatial.transform import Rotation as R

In [29]:
def rotm2Euler(x):
    """Rotation matrix to Euler Angles XYZ intrinsic vector for Shrimp"""
    r = R.from_dcm(x)
    return r.as_euler('XYZ')
    
def euler2Rotm(x):
    """Euler angles XYZ intrinsic vector to Rotation Matrix for Shrimp"""
    r = R.from_euler('XYZ', x)
    return r.as_dcm()

## Relative Wind
Calculating the relative wind is pretty similar to the blade equations above. 
We want to take the given blade section, and then calculate the relative wind vector in the propellor frame. 

![relativewind](../lib/relative_wind.png)

In [30]:
def getRelativeWind(bladeIndex, totalNumBlades, bladeRadius, vel_cg2w_w, 
                    rot_flyer2world, angvel_b2w_w, inflowVelocity, height_body2element,
                    prop_angvel):
    """
    Returns relative wind angle as well as the wind angle vector for 
    the given quantities
    
    Return values:
    relativeWind_e : the relative wind in the propellor frame
    relWindAngleGamma: the wind angle relative to the propellor blade disk angle
    
    
    Arguements:
    bladeIndex: the index of the blade to be used. Assumes blade 0 is aligned with the 
    x-axis of the body
    totalNumBlades: total number of blades, assuming equal spacing
    bladeRadius: the radius to do this overall calculation
    vel_cg2w_w: velocity of the cg relative to the world in the world frame
    rot_flyer2world: DCM of flyer to world 
    angvel_b2w_w: angular velocity of body relative to world in the world frame 
    inflowVelocity: also called nu - scalar amount of inflow velocity
    height_body2element: the distance from cg to element along the body z axis 
    prop_angvel: propellor angular velocity (spin rate)
    
    Notation:
    _e is prop element frame
    _f is flyer frame
    _w is world/inertial frame
    
    TODO: clean up cg vs b 
    """

    # Inflow velocity is only coming from the top of the disc, so 
    # it is always in the positive z direction 
    vel_inducedInflow_e = np.array([0, 0, inflowVelocity])
    
    
    # Velocity of the flyer from the center of gravity moving forward
    # which is simply the negative of the cg velocity in the flyer frame 
    euler_flyer2world = rotm2Euler(rot_flyer2world)
    # Get yaw from euler angles of flyer from world
    yaw = euler_flyer2world[2] # 3rd angle is yaw 
    propellor_angle = bladeIndex/totalNumBlades*2*np.pi + yaw
    
    rot_flyer2prop = R.from_euler('Z', propellor_angle).as_dcm()
    
    rot_world2flyer = np.transpose(rot_flyer2world)
    vel_flyer2world_f = rot_world2flyer.dot(vel_cg2w_w)
    vel_flyer2world_e = rot_flyer2prop.dot(vel_flyer2world_f)

    
    # Velocity from rotational component
    # First calculate wxr=v in the flyer frame 
    # Then rotate v to the propellor frame 
    angvel_b2w_f = rot_world2flyer.dot(angvel_b2w_w).dot(rot_flyer2world)
    
    pos_x_blade = bladeRadius*np.sin(propellor_angle)
    pos_y_blade = bladeRadius*np.cos(propellor_angle)
    # I think sin and cos should be switched, but seems to work
    # like this. TODO: figure out. 
    r_b2element_f = np.array([pos_x_blade, pos_y_blade, height_body2element])
    
    
    angvel_spin_f = np.array([0, 0, prop_angvel])
    # Total angular velocity of the propellor is body + prop
    angvel_prop_f = angvel_b2w_f + angvel_spin_f
    vel_spin_f = np.cross(angvel_prop_f, r_b2element_f)
    vel_spin_e = rot_flyer2prop.dot(vel_spin_f)
    
    # Sum all forces together
    relativeWind_e = -(vel_flyer2world_e + vel_inducedInflow_e + vel_spin_e)
    relWindAngleGamma = np.arctan2(relativeWind_e[2], relativeWind_e[0])

    return (relativeWind_e, relWindAngleGamma, r_b2element_f, rot_flyer2prop)

In [31]:
def testRelativeWind():
    # Test for relative wind 
    flyerEulerAngles_w = np.pi/180*np.array([0,0,0])
    # Below is technically body2world, but no big deal
    rot_flyer2world = euler2Rotm(flyerEulerAngles_w) 

    totalNumBlades = 2
    bladeRadius = 0.1
    bladeIndex = 1
    vel_cg2w_w = np.array([10, 0, 0])
    angvel_b2w_w = np.array([0.0, 0.0, 50.0])
    inflowVelocity = 2 # TODO nu
    height_element2ref = 0.1 # TODO
    prop_angvel = 10

    # Using XYZ euler angles such that the last angle Z is the correct psi angle
    (relWind, gamma, _, _) = getRelativeWind(bladeIndex, totalNumBlades, bladeRadius, vel_cg2w_w, 
                        rot_flyer2world, angvel_b2w_w, inflowVelocity, height_element2ref, 
                                        prop_angvel)

    print('gamma', gamma*180/np.pi)
    print('relwind', relWind)

testRelativeWind()

gamma -7.125016348901798
relwind [ 1.6000000e+01 -1.2246468e-15 -2.0000000e+00]


## Lift and Drag
Note that lift is POSITIVE in this model. Z is up. 
$$L = \frac{1}{2} \rho V^2 C_L S$$
$$D = \frac{1}{2} \rho V^2 C_D S$$
Where 
* $C_L, C_D$ are lift and drag coefficients, functions of angle of attack ($alpha$)
* $S$ is wing area. 
* $\rho$ is air density 
* $V$ is velocity at the wing element

In [32]:
def getAngleOfAttack(bladePitch, relWindAngleGamma):
    """ Returns angle of attack between -pi and pi"""
    unwrapped_angle = bladePitch + relWindAngleGamma
    return (unwrapped_angle + np.pi) % (2 * np.pi ) - np.pi

def clFlatPlate(alpha):
    """ Coeff of lift for a Flat Plate """
    return 2.*np.sin(alpha)*np.cos(alpha)


def cdFlatPlate(alpha):
    """ Coeff of drag for a Flat Plate """
    return 2.*np.sin(alpha)**2.0


def cmFlatPlate(alpha):
    """ Coeff of pitch moment for a Flat Plate """
    return 0.0

def getLiftDragElement(elementWidth, chord, rho, relWind_e, alpha):
    """ Returns magnitude of lift and drag
    Arguements:
    elementWidth: the width of the blade section 
    chord: airfoil chord
    rho: air density
    relWind_e : relative wind in the propellor frame
    alpha: angle of attack of section
    Returns:
    (lift, drag) scalars
    """
    # velocity only incorporates the x and z components in the wind frame 
    speed = relWind_e.dot([1,0,1])    
    coeffLift = clFlatPlate(alpha)
    coeffDrag = cdFlatPlate(alpha)
    quantity = 0.5*speed*speed*chord*elementWidth*rho
    lift = quantity * coeffLift
    drag = quantity * coeffDrag
    return (lift, drag)


In [33]:
def rpm2RadiansPerSecond(x:float) -> float:
     return 0.10472 * x

In [34]:
class BladeElementParameters:
    def __init__(self, rho, pitch, width, radius, chord, index, totalNumBlades, 
                 height_body2element, inflowVel, spinVel):
        self.rho = rho
        self.pitch = pitch 
        self.width = width  # this is in the radial direction (strip model width)
        self.radius = radius  # this should probably "middle" of the overall radius width 
        self.chord = chord
        self.index = index # Blade Index out of n blades
        self.totalNumBlades = totalNumBlades # total number of blades. A normal prop has 2
        self.height_body2element = height_body2element # scalar [m]
        self.inflowVel = inflowVel # Inflow velocity for this element. Scalar [m/s]
        self.spinVel = spinVel # spin velocity for this element. Scalar [m/s]

class ShrimpState:
    """ Shrimp state object. Stores state in a clear format, and also converts 
        the clear format to a numpy vector for ODE integration.
    """
    def __init__(self, r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel):
        self.r_b2w_w = r_b2w_w  # np.array 3x1 position vector from world origin to body/cg
        self.vel_b2w_w = vel_b2w_w  # np.array 3x1 velocity vector from world origin to body/cg
        self.euler_b2w_w = euler_b2w_w  # np.array 3x1 euler angles vector from world to body frame
        self.angvel_b2w_w = angvel_b2w_w  # np.array 3x1 euler velocity vector from world to body frame
        self.inflowVel = inflowVel # scalar inflow velocity adjustment due to duct on the shaft propellor [m/s]
        self.prop_angVel = prop_angVel # the speed of the spin of the propellor. Scalar [rad/s]

    def toStateVec(self):
        raise Exception('TODO: implement')
    
    
# Blade equations 
def getElementForceMoment(bladeParams, state):
    """Returns force and moment vectors in the flyer frame 
    Assumes the element radius is the radius at the center of the width. 
    The width is only used in a surface area calculation. 
    Arguments:
        bladeParams (BladeElementParameters)
        state (ShrimpState)
    """
    
    rho = bladeParams.rho
    bladeIndex = bladeParams.index
    bladeRadius = bladeParams.radius
    vel_b2w_w = state.vel_b2w_w
    angvel_b2w_w = state.angvel_b2w_w
    inflowVelocity = bladeParams.inflowVel
    height_body2element = bladeParams.height_body2element
    prop_angvel = bladeParams.spinVel
    euler_b2w = state.euler_b2w_w
    totalNumBlades = bladeParams.totalNumBlades
    
    rot_body2world = euler2Rotm(euler_b2w)
    # TODO: should be rot_flyer2world, need to iron this out 
    # more explicitly and write it up
    (relativeWind_e, gamma, r_b2Element_f, rot_flyer2prop) = getRelativeWind(bladeIndex, totalNumBlades, 
                                                                              bladeRadius, vel_b2w_w, 
                                                                              rot_body2world, angvel_b2w_w, 
                                                                              inflowVelocity, height_body2element,
                                                                              prop_angvel)
    
    alpha = getAngleOfAttack(gamma, bladeParams.pitch)
    (lift, drag) = getLiftDragElement(bladeParams.width, bladeParams.chord, rho, relativeWind_e, alpha)
    
    # Forces in the propellor frame
    normalForce_e = lift*np.cos(gamma) + drag*np.sin(gamma)
    tangentialForce_e = -lift*np.sin(gamma) + drag*np.cos(gamma)
    
    # Z-axis is up! 
    forces_e = np.array([-tangentialForce_e, 0, normalForce_e])
    
    rot_prop2flyer = np.transpose(rot_flyer2prop)
    forces_f = rot_prop2flyer.dot(forces_e)
    moments_f = np.cross(forces_f, r_b2Element_f)
    print('Element forces', forces_f)
    return(forces_f, moments_f)
    
def testBladeElement(radius, chord, pitch, width):
    rho = 1.225
    bladeIndex = 1
    totalNumBlades = 2
    height_body2element = 0.01
    
    inflowVel = 1
    prop_angVel = rpm2RadiansPerSecond(500)
    # Set up this particular propellor element
    bladeParams = BladeElementParameters(rho, pitch, width, radius, chord, bladeIndex,
                                          totalNumBlades, height_body2element, inflowVel, prop_angVel)
    
    r_b2w_w = np.array([0.1, 0.1, 0.1])
    vel_b2w_w = np.array([0.1, 0, 0])
    euler_b2w_w = np.array([10, 0, 40]) * np.pi/180.
    angvel_b2w_w = np.array([0.01, 0, 0.02])
    inflowVel = 1
    prop_angVel = rpm2RadiansPerSecond(500)
    state = ShrimpState(r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel)
    
    (forces_f, moments_f) = getElementForceMoment(bladeParams, state) 
    return (forces_f, moments_f)
    
def testBlade():
    n = 5 # The number of sections to split the blade into for element theory
    radiusRoot = 0
    radiusTip = 0.1 # maximum blade radius / tip to root
    width = (radiusTip-radiusRoot)/n
    chordRoot = 0.1 # Chord at the root of the blade
    chordTip = 0.2 # Chord at the tip of the blade
    pitchRoot = 0*np.pi/180
    pitchTip = 15*np.pi/180
    
    radii = np.linspace(radiusRoot, radiusTip, n)
    chords = np.linspace(chordRoot, chordTip, n)
    pitches = np.linspace(pitchRoot, pitchTip, n)
    
    forceMomentTuples = [testBladeElement(r, c, p, width) for (r,c,p) in zip(radii, chords, pitches)]

    forces = sum([f for f,_ in forceMomentTuples])
    moments = sum([m for _,m in forceMomentTuples])
    print('Forces', forces)
    print('Moments', moments)
testBlade()

Element forces [ 2.07636762e-20 -1.74227931e-20 -1.97466460e-03]
Element forces [-1.34974527e-05  1.13257076e-05 -2.68824264e-04]
Element forces [-0.00023855  0.00020016 -0.00236532]
Element forces [-0.00027502  0.00023077 -0.00180486]
Element forces [ 0.00140312 -0.00117736  0.0068358 ]
Forces [ 0.00087606 -0.0007351   0.00042213]
Moments [ 0.00031686 -0.00028081 -0.00014023]


In [ ]:
def renderShrimp(states, times):
    """ Makes a 3D rendering of ShrimpStates at given times
        Argumenets:
            states: a list of shrimp states corresponding to given times
            times: a np.ndarray of times to plot at
    """
    
    # Allen Code here 